In [1]:
#Please select GPU first (from Edit->NotebookSetting)
import pandas as pd
import numpy as np
import io
import os
import re
from google.colab import drive
drive.mount('/content/drive')
!pip install fastai==2.3.1
from fastai.text.all import *

Mounted at /content/drive
     |████████████████████████████████| 204kB 27.4MB/s 
     |████████████████████████████████| 804.1MB 22kB/s 
     |████████████████████████████████| 61kB 19kB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.8.1 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.8.1 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [3]:
def generate_target(data):
    tmp = data.copy()
    tmp.set_index(['ID','TITLE','ABSTRACT'],inplace=True)
    tmp['LABEL'] = tmp.idxmax(axis=1)
    tmp = tmp.reset_index()
    tmp = tmp[['ID','TITLE','ABSTRACT','LABEL']]
    return tmp
path = '/content/drive/MyDrive/colab_data'
data = pd.read_csv(os.path.join(path, "train.csv"))
data = generate_target(data)
convert_label = {'Computer Science':0,'Physics':1,'Mathematics':2,'Statistics':3,'Quantitative Biology':4,'Quantitative Finance':5}
data['convert_label'] = data.LABEL.map(convert_label)


In [4]:
df_clas = data[['TITLE', 'LABEL']].dropna(subset=['LABEL'])
df_clas.columns = df_clas.columns.str.strip()

In [6]:
dls_lm = TextDataLoaders.from_df(data, text_col='TITLE', is_lm=True, valid_pct=0.1)
dls_lm.show_batch(max_n=2)
# torch.save(dls_lm, os.path.join(path, 'dls_lm.pkl'))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,xxbos xxmaj variable xxmaj exponent xxmaj fock xxmaj spaces xxbos a partial inverse problem for the sturm - liouville operator on the graph with a loop xxbos xxmaj stein xxmaj variational xxmaj message xxmaj passing for xxmaj continuous xxmaj graphical xxmaj models xxbos xxmaj deriving mesoscopic models of collective behaviour for finite populations xxbos xxmaj realistic finite temperature simulations of magnetic systems using quantum statistics xxbos xxmaj geometric vulnerability of xxunk institutions,xxmaj variable xxmaj exponent xxmaj fock xxmaj spaces xxbos a partial inverse problem for the sturm - liouville operator on the graph with a loop xxbos xxmaj stein xxmaj variational xxmaj message xxmaj passing for xxmaj continuous xxmaj graphical xxmaj models xxbos xxmaj deriving mesoscopic models of collective behaviour for finite populations xxbos xxmaj realistic finite temperature simulations of magnetic systems using quantum statistics xxbos xxmaj geometric vulnerability of xxunk institutions against
1,xxbos xxmaj drone xxmaj xxunk xxmaj optimization : a xxmaj self - adaptive xxmaj algorithm for xxmaj global xxmaj numerical xxmaj optimization xxbos a xxmaj review of laser - plasma xxmaj ion xxmaj acceleration xxbos xxmaj determinants of xxunk - xxunk kinetics of short xxup dna with xxunk ends xxbos xxmaj joins in the strong xxmaj xxunk degrees xxbos xxmaj learning xxmaj program xxmaj component xxmaj order xxbos xxmaj analysis of error,xxmaj drone xxmaj xxunk xxmaj optimization : a xxmaj self - adaptive xxmaj algorithm for xxmaj global xxmaj numerical xxmaj optimization xxbos a xxmaj review of laser - plasma xxmaj ion xxmaj acceleration xxbos xxmaj determinants of xxunk - xxunk kinetics of short xxup dna with xxunk ends xxbos xxmaj joins in the strong xxmaj xxunk degrees xxbos xxmaj learning xxmaj program xxmaj component xxmaj order xxbos xxmaj analysis of error control


In [7]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult = 0.3, metrics=[accuracy, Perplexity()]).to_fp16()

In [8]:
learn.unfreeze()
learn.fit_one_cycle(4, 1e-3) 

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.461620,4.121613,0.383545,61.658619,00:21
1,4.076141,3.885459,0.400906,48.689270,00:21
2,3.826846,3.809280,0.406603,45.117939,00:21
3,3.684431,3.803184,0.408583,44.843754,00:21


In [9]:
TEXT = "Apple"
N_WORDS = 45
N_SENTENCES = 2
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Xxunk Energy Preferences in Quasi - equilibrium Systems Infinite Pt - flat Column Functions Learning Semi - supervised Learning for Special Session Learning via Task Learning
Xxunk : a Tactile Neural Network for Images and Evaluating Visual Stimulus Tracking Novel Structural Testing for Accelerator Systems Some Hierarchies of High Dimensional Semiconductor


In [11]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df_clas, test_size=0.33, random_state=42)

In [13]:
dls_clas = DataBlock(
    blocks = (TextBlock.from_df('TITLE', seq_len = dls_lm.seq_len, vocab = dls_lm.vocab), CategoryBlock),
    get_x = ColReader('text'),
    get_y = ColReader('LABEL'),
    splitter = RandomSplitter()
).dataloaders(df_train, bs = 64)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [16]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()

In [17]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.262770,1.029961,0.683986,00:17
1,0.983322,0.877945,0.708541,00:17
2,0.873867,0.814948,0.719929,00:17
3,0.798573,0.778062,0.729181,00:17
4,0.776375,0.773119,0.730961,00:17


In [18]:
pred_dl = dls_clas.test_dl(df_val['TITLE'])
preds = learn.get_preds(dl=pred_dl)

In [25]:
learn.dls.vocab[1]

['Computer Science', 'Mathematics', 'Physics', 'Quantitative Biology', 'Quantitative Finance', 'Statistics']

In [19]:
preds

(tensor([[8.8741e-01, 1.4511e-03, 6.3268e-04, 1.6303e-02, 4.6932e-03, 8.9511e-02],
         [7.6366e-01, 4.2675e-02, 9.0572e-03, 2.4995e-02, 1.4361e-02, 1.4525e-01],
         [5.9968e-01, 8.2334e-02, 1.2075e-02, 1.8849e-02, 3.3389e-02, 2.5367e-01],
         ...,
         [6.0792e-03, 6.5049e-01, 2.7436e-01, 2.5846e-02, 2.7857e-02, 1.5365e-02],
         [2.1029e-03, 1.2174e-01, 8.5334e-01, 1.1349e-02, 9.9274e-03, 1.5385e-03],
         [7.6692e-01, 2.7582e-02, 1.1154e-03, 2.6767e-02, 5.1935e-02, 1.2569e-01]]),
 None)

In [26]:
# convert_label = {'Computer Science':0,'Physics':1,'Mathematics':2,'Statistics':3,'Quantitative Biology':4,'Quantitative Finance':5}
# data['convert_label'] = data.LABEL.map(convert_label)
cols = ['Computer Science', 'Mathematics', 'Physics', 'Quantitative Biology', 'Quantitative Finance', 'Statistics']
predsTest = pd.DataFrame(np.asarray(preds[0]), columns = cols)
predsTest['pred'] = predsTest.idxmax(axis = 1)
# predsTest['pred_convert'] = predsTest['pred'].apply(lambda x: target_map[x])

In [27]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(predsTest.pred, df_val.LABEL)

array([[2502,  342,  230,   93,   35,  479],
       [ 169,  972,  166,    5,    7,   37],
       [ 101,  119, 1470,   36,    8,   34],
       [   4,    0,   12,   19,    1,    3],
       [   3,    0,    2,    1,   10,    2],
       [  20,   13,    6,    3,    3,   14]])

In [28]:
print(classification_report(predsTest.pred, df_val.LABEL))

                      precision    recall  f1-score   support

    Computer Science       0.89      0.68      0.77      3681
         Mathematics       0.67      0.72      0.69      1356
             Physics       0.78      0.83      0.80      1768
Quantitative Biology       0.12      0.49      0.19        39
Quantitative Finance       0.16      0.56      0.24        18
          Statistics       0.02      0.24      0.04        59

            accuracy                           0.72      6921
           macro avg       0.44      0.58      0.46      6921
        weighted avg       0.81      0.72      0.75      6921

